<a href="https://kaggle.com/kernels/welcome?src=https://github.com/yuval-alaluf/restyle-encoder/blob/main/notebooks/inference_playground.ipynb"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a><a href="https://colab.research.google.com/github/yuval-alaluf/restyle-encoder/blob/main/notebooks/inference_playground.ipynb"><img align="left" title="Open in Colab" src="https://colab.research.google.com/assets/colab-badge.svg"></a>

In [1]:
import os
# os.chdir('/content')
CODE_DIR = 'restyle-encoder-light'

In [2]:
!git clone https://github.com/RahulBhalley/restyle-encoder-light.git $CODE_DIR

In [3]:
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [4]:
os.chdir(f'./{CODE_DIR}')

In [21]:
from argparse import Namespace
import time
import os
import sys
import pprint
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp
from models.e4e import e4e

%load_ext autoreload
%autoreload 2

## Step 1: Select Experiment Type
Select which experiment you wish to perform inference on:

In [22]:
#@title Select which experiment you wish to perform inference on: { run: "auto" }
experiment_type = 'horse_encode' #@param ['ffhq_encode', 'cars_encode', 'church_encode', 'horse_encode', 'afhq_wild_encode', 'toonify']

## Step 2: Prepare to Download Pretrained Models 
As part of this repository, we provide pretrained models for each of the above experiments. Here, we'll create the download command needed for downloading the desired model.

Note: in this notebook, we'll be using ReStyle applied over pSp for all domains except for the horses domain where we'll be using e4e. This is done since e4e is generally able to generate more realistic reconstructions on this domain. 

In [23]:
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory ../pretrained_models. """
    current_directory = os.getcwd()
    save_path = os.path.join(os.path.dirname(current_directory), CODE_DIR, "pretrained_models")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url    

In [24]:
MODEL_PATHS = {
    "ffhq_encode": {"id": "1sw6I2lRIB0MpuJkpc8F5BJiSZrc0hjfE", "name": "restyle_psp_ffhq_encode.pt"},
    "cars_encode": {"id": "1zJHqHRQ8NOnVohVVCGbeYMMr6PDhRpPR", "name": "restyle_psp_cars_encode.pt"},
    "church_encode": {"id": "1bcxx7mw-1z7dzbJI_z7oGpWG1oQAvMaD", "name": "restyle_psp_church_encode.pt"},
    "horse_encode": {"id": "19_sUpTYtJmhSAolKLm3VgI-ptYqd-hgY", "name": "restyle_e4e_horse_encode.pt"},
    "afhq_wild_encode": {"id": "1GyFXVTNDUw3IIGHmGS71ChhJ1Rmslhk7", "name": "restyle_psp_afhq_wild_encode.pt"},
    "toonify": {"id": "1GtudVDig59d4HJ_8bGEniz5huaTSGO_0", "name": "restyle_psp_toonify.pt"}
}

path = MODEL_PATHS[experiment_type]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"]) 

## Step 3: Define Inference Parameters

Below we have a dictionary defining parameters such as the path to the pretrained model to use and the path to the image to perform inference on.  
While we provide default values to run this script, feel free to change as needed.

In [25]:
EXPERIMENT_DATA_ARGS = {
    "ffhq_encode": {
        "model_path": "pretrained_models/restyle_psp_ffhq_encode.pt",
        "image_path": "notebooks/images/face_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
    "cars_encode": {
        "model_path": "pretrained_models/restyle_psp_cars_encode.pt",
        "image_path": "notebooks/images/car_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((192, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
    "church_encode": {
        "model_path": "pretrained_models/restyle_psp_church_encode.pt",
        "image_path": "notebooks/images/church_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
    "horse_encode": {
        "model_path": "pretrained_models/restyle_e4e_horse_encode.pt",
        "image_path": "notebooks/images/horse_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
    "afhq_wild_encode": {
        "model_path": "pretrained_models/restyle_psp_afhq_wild_encode.pt",
        "image_path": "notebooks/images/afhq_wild_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
    "toonify": {
        "model_path": "pretrained_models/restyle_psp_toonify.pt",
        "image_path": "notebooks/images/toonify_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
}

In [26]:
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[experiment_type]

To reduce the number of requests to fetch the model, we'll check if the model was previously downloaded and saved before downloading the model.  
We'll download the model for the selected experiment and save it to the folder `../pretrained_models`.

We also need to verify that the model was downloaded correctly. All of our models should weigh approximately 800MB - 1GB.  
Note that if the file weighs several KBs, you most likely encounter a "quota exceeded" error from Google Drive. In that case, you should try downloading the model again after a few hours.

In [27]:
if not os.path.exists(EXPERIMENT_ARGS['model_path']) or os.path.getsize(EXPERIMENT_ARGS['model_path']) < 1000000:
    print(f'Downloading ReStyle model for {experiment_type}...')
    os.system(f"wget {download_command}")
    # if google drive receives too many requests, we'll reach the quota limit and be unable to download the model
    if os.path.getsize(EXPERIMENT_ARGS['model_path']) < 1000000:
        raise ValueError("Pretrained model was unable to be downloaded correctly!")
    else:
        print('Done.')
else:
    print(f'ReStyle model for {experiment_type} already exists!')

## Step 4: Load Pretrained Model
We assume that you have downloaded all relevant models and placed them in the directory defined by the above dictionary.

In [49]:
model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')

In [50]:
opts = ckpt['opts']
pprint.pprint(opts)

In [51]:
# update the training options
opts['checkpoint_path'] = model_path
# opts['encoder_type'] = 'ResNetProgressiveBackboneEncoderLight'
opts['device'] = 'cpu'

In [52]:
opts = Namespace(**opts)
if experiment_type == 'horse_encode': 
    net = e4e(opts)
else:
    net = pSp(opts)
    
net.eval()
# net.cuda()
print('Model successfully loaded!')

In [53]:
type(net.encoder)

In [54]:
type(net.decoder)

In [55]:
x = torch.randn(1, 6, 1024, 1024)
encoder_trace = torch.jit.trace(net.encoder, x)

In [56]:
torch.jit.save(encoder_trace, 'ResNetProgressiveBackboneEncoder_trace.pt')

## Step 5: Visualize Input

In [ ]:
image_path = EXPERIMENT_DATA_ARGS[experiment_type]["image_path"]
original_image = Image.open(image_path).convert("RGB")

In [ ]:
if experiment_type == 'cars_encode':
    original_image = original_image.resize((192, 256))
else:
    original_image = original_image.resize((256, 256))

In [ ]:
original_image

### Align Image

Note: in this notebook we'll run alignment on the input image when working on the human facial domain.

In [ ]:
def run_alignment(image_path):
    import dlib
    from scripts.align_faces_parallel import align_face
    if not os.path.exists("shape_predictor_68_face_landmarks.dat"):
        print('Downloading files for aligning face image...')
        os.system('wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2')
        os.system('bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2')
        print('Done.')
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    aligned_image = align_face(filepath=image_path, predictor=predictor) 
    print("Aligned image has shape: {}".format(aligned_image.size))
    return aligned_image 

In [ ]:
if experiment_type in ['ffhq_encode', 'toonify']:
    input_image = run_alignment(image_path)
else:
    input_image = original_image

In [ ]:
input_image.resize((256, 256))

## Step 6: Perform Inference

In [ ]:
img_transforms = EXPERIMENT_ARGS['transform']
transformed_image = img_transforms(input_image)

Before running inference, we need to generate the image corresponding to the average latent code. These will be used to initialize the iterative refinement process.

In [ ]:
def get_avg_image(net):
    avg_image = net(net.latent_avg.unsqueeze(0),
                    input_code=True,
                    randomize_noise=False,
                    return_latents=False,
                    average_code=True)[0]
    avg_image = avg_image.to('cuda').float().detach()
    if experiment_type == "cars_encode":
        avg_image = avg_image[:, 32:224, :]
    return avg_image

Now we'll run inference. By default, we'll run using 5 inference steps. You can change the parameter in the cell below.

In [ ]:
opts.n_iters_per_batch = 5
opts.resize_outputs = False  # generate outputs at full resolution

In [ ]:
from utils.inference_utils import run_on_batch

with torch.no_grad():
    avg_image = get_avg_image(net)
    tic = time.time()
    result_batch, result_latents = run_on_batch(transformed_image.unsqueeze(0).cuda(), net, opts, avg_image)
    toc = time.time()
    print('Inference took {:.4f} seconds.'.format(toc - tic))

### Visualize Result

We'll visualize the step-by-step outputs side by side.

In [ ]:
if opts.dataset_type == "cars_encode":
    resize_amount = (256, 192) if opts.resize_outputs else (512, 384)
else:
    resize_amount = (256, 256) if opts.resize_outputs else (opts.output_size, opts.output_size)

In [ ]:
def get_coupled_results(result_batch, transformed_image):
    """
    Visualize output images from left to right (the input image is on the right)
    """
    result_tensors = result_batch[0]  # there's one image in our batch
    result_images = [tensor2im(result_tensors[iter_idx]) for iter_idx in range(opts.n_iters_per_batch)]
    input_im = tensor2im(transformed_image)
    res = np.array(result_images[0].resize(resize_amount))
    for idx, result in enumerate(result_images[1:]):
        res = np.concatenate([res, np.array(result.resize(resize_amount))], axis=1)
    res = np.concatenate([res, input_im.resize(resize_amount)], axis=1)
    res = Image.fromarray(res)
    return res

Note that the step-by-step outputs are shown left-to-right with the original input on the right-hand side.

In [ ]:
res = get_coupled_results(result_batch, transformed_image)
res

In [ ]:
# save image 
res.save(f'./{experiment_type}_results.jpg')

# Encoder Bootstrapping

In the paper, we introduce an encoder bootstrapping technique that can be used to solve the image toonification task by pairing an FFHQ-based encoder with a Toon-based encoder.  

We demonstrate this idea below.

In [ ]:
# download the ffhq-based encoder if not previously downloaded
path = MODEL_PATHS['ffhq_encode']
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS['ffhq_encode']
ffhq_model_path = EXPERIMENT_ARGS['model_path']
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"]) 
if not os.path.exists(ffhq_model_path):
    print('Downloading FFHQ ReStyle encoder...')
    os.system(f"wget {download_command}")
    print('Done.')
else:
    print('FFHQ ReStyle encoder already exists!')

In [ ]:
# download the toon-based encoder if not previously downloaded
path = MODEL_PATHS['toonify']
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS['toonify']
toonify_model_path = EXPERIMENT_ARGS['model_path']
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"]) 
if not os.path.exists(toonify_model_path):
    print('Downloading Toonify ReStyle encoder...')
    os.system(f"wget {download_command}")
    print('Done.')
else:
    print('Toonify ReStyle encoder already exists!')

In [ ]:
# load models 
ckpt = torch.load(ffhq_model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = ffhq_model_path
opts = Namespace(**opts)
net1 = pSp(opts)
net1.eval()
net1.cuda()
print('FFHQ Model successfully loaded!')

ckpt = torch.load(toonify_model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = toonify_model_path
opts = Namespace(**opts)
net2 = pSp(opts)
net2.eval()
net2.cuda()
print('Toonify Model successfully loaded!')

In [ ]:
# load image 
image_path = EXPERIMENT_DATA_ARGS['toonify']["image_path"]
original_image = Image.open(image_path).convert("RGB")

In [ ]:
# transform image
img_transforms = EXPERIMENT_ARGS['transform']
transformed_image = img_transforms(original_image)

In [ ]:
opts.n_iters_per_batch = 5
opts.resize_outputs = False  # generate outputs at full resolution

In [ ]:
from scripts.encoder_bootstrapping_inference import run_on_batch

with torch.no_grad():
    avg_image = get_avg_image(net1)
    tic = time.time()
    result_batch = run_on_batch(transformed_image.unsqueeze(0).cuda(), net1, net2, opts, avg_image)
    toc = time.time()
    print('Inference took {:.4f} seconds.'.format(toc - tic))

Again we'll visualize the results from left to right. Here, the leftmost image is the inverted FFHQ image that is used to initialize the toonify ReStyle encoder. The following images show iterative results outputted by the toonify model.
Finally, the rightmost image is the original input image.

In [ ]:
res = get_coupled_results(result_batch, transformed_image)
res

In [ ]:
# save image 
res.save(f'./encoder_bootstrapping_results.jpg')